In [48]:
import os
import json
import pandas as pd
import traceback

In [49]:
from langchain.chat_models import ChatOpenAI

In [50]:
from dotenv import load_dotenv

In [51]:
load_dotenv()

True

In [52]:
KEY=os.getenv('OPENAI_API_KEY')


In [53]:
print(KEY)

sk-proj-5NZfTVoUyRVL97quyrQzT3BlbkFJ2o04IfbwoAzDSnEj4YGB


In [54]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

In [55]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import  SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [56]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [57]:

TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Make mcq that way that the options in the mcqs are not identical to each other.
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}"""


In [58]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [59]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [60]:

TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [61]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [62]:

review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [63]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [64]:
file_path = r"/home/seaflux/MCQ-Generator/data.txt"


In [65]:
with open(file_path, 'r') as f:
    TEXT = f.read()

In [66]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [67]:

NUMBER=3 
SUBJECT="AI"
TONE="simple"

In [68]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and uses learning and intelligence to take actions that maximize their chances of achieving defined goals.[1] Such machines may be called AIs.

AI technology is widely used throughout industry, government, and science. Some high-profile applications include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go).[2] However, man

In [69]:

print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2167
Prompt Tokens:1710
Completion Tokens:457
Total Cost:0.0034790000000000003


In [70]:

quiz=response.get("quiz")

In [71]:
quiz=json.loads(quiz)


In [72]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [73]:
quiz_table_data

[{'MCQ': 'What was the first person to conduct substantial research in the field of machine intelligence?',
  'Choices': 'a: Alan Turing | b: Elon Musk | c: Jeff Bezos | d: Mark Zuckerberg',
  'Correct': 'a'},
 {'MCQ': 'Which of the following is NOT a high-profile application of AI technology?',
  'Choices': 'a: Advanced web search engines | b: Autonomous vehicles | c: Social media influencers | d: Generative and creative tools',
  'Correct': 'c'},
 {'MCQ': 'What is one of the long-term goals of AI research?',
  'Choices': 'a: Creating robots with emotions | b: Achieving general intelligence | c: Building time machines | d: Developing teleportation devices',
  'Correct': 'b'}]

In [74]:
quiz=pd.DataFrame(quiz_table_data)

In [75]:

quiz.to_csv("machinelearning.csv",index=False)

,MCQ,Choices,Correct
0,What was the first person to conduct substanti...,a: Alan Turing | b: Elon Musk | c: Jeff Bezos ...,a
1,Which of the following is NOT a high-profile a...,a: Advanced web search engines | b: Autonomous...,c
2,What is one of the long-term goals of AI resea...,a: Creating robots with emotions | b: Achievin...,b


In [76]:

from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'05_22_2024_15_27_30'